## Latihan Penanganan Missing Value dg dataset Titanic: teknik Substitusi


import library Python yang diperlukan untuk analisi data: Numpy dan Panda 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


### Load data dan memahami data

Untuk secara detil analisis eksplorasi data secara singkat, dapat melihat ke Notebook titanic00_EDA


In [2]:
df = pd.read_csv('../dataset/titanic/train.csv')
dft = df.copy()

### Teknik 1. Penanganan _Missing value_ dengan **Substitusi**
Mengganti _missing value_ dengan nilai tertentu seperti NA atau Unknown.

Berdasarkan hasil analisi data di sebelumnya, dapat diketahui bahwa: 
* field/kolom/atribut **PassengerId** karena hanya sebagai nomor urut atau unique variabel untuk setiap record, tidak kita perlukan. 
* field/kolom/atribut **Name** memiliki jumlah **unique** sebanyak jumlah record total, yaitu 891, sehingga tidak signifikan untuk pemodelan prediksi.
* field/kolom/atribut **Ticket** memiliki jumlah **unique** hampir sebanyak jumlah record total, yaitu 681 (85%), sehingga tidak signifikan untuk pemodelan prediksi.
* ada 3 field yang memiliki _missing value_ yaitu: age, cabin dan embarked. 

Kemudian untuk record dengan _missing value_ yang akan kita gantikan dengan mean, median, nilai paliang banyak muncul (most_frequent) adalah atribute sebagai berikut:
**Cabin, Age dan Embarked**. Kita akan menggunakan library SimpleImputer untuk mengimplementasikan substitusi ini. 

In [3]:
dft.drop(["PassengerId","Name","Ticket"], axis=1, inplace=True)

In [4]:
dft.describe(include = "all")

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
count,891.000000,891.000000,891,714.000000,891.000000,891.000000,891.000000,204,889
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,147,3
top,NaN,NaN,male,NaN,NaN,NaN,NaN,B96 B98,S
freq,NaN,NaN,577,NaN,NaN,NaN,NaN,4,644
mean,0.383838,2.308642,NaN,29.699118,0.523008,0.381594,32.204208,NaN,NaN
std,0.486592,0.836071,NaN,14.526497,1.102743,0.806057,49.693429,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN
25%,0.000000,2.000000,NaN,20.125000,0.000000,0.000000,7.910400,NaN,NaN
50%,0.000000,3.000000,NaN,28.000000,0.000000,0.000000,14.454200,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,0.000000,31.000000,NaN,NaN


In [5]:
dft.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


In [7]:
dft.columns = ['Survived','Pclass', 
               'Sex', 'Age',
               'SibSp','Parch',
               'Fare', 'Cabin',
              'Embarked']

In [8]:
#
# Missing values direpresentatikan dengan NaN sehingga terspesifikasi. 
# Jika dia adalah data kosong, maka missing value terspesifikasi 
# sebagai ''
imputer = SimpleImputer(missing_values=np.NaN, strategy='median')
 
dft.Age = imputer.fit_transform(dft['Age'].values.reshape(-1,1))[:,0]


In [9]:
#
# Missing values direpresentatikan dengan NaN sehingga terspesifikasi. 
# Jika dia adalah data kosong, maka missing value terspesifikasi 
# sebagai ''
imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
 
dft.Cabin = imputer.fit_transform(dft['Cabin'].values.reshape(-1,1))[:,0]
dft.Embarked = imputer.fit_transform(dft['Embarked'].values.reshape(-1,1))[:,0]


In [10]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     891 non-null    object 
 8   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


### Pemodelan dengan data dengan algoritma Decision Tree

#### Seleksi Fitur (_Feature Selection_)
Sebagaimana algoritma _supervised_ lainnya, kita perlu membagi atribut menjadi 2 bagian. Satu bagian terdiri dari 1 atribut, sebagai variabel _**target**_, dan sisanya adalah sebagai variabel _**independent**_. 

Dalam hal ini, atribute **Survived** adalah variabel target, dan sisanya adalah variabel independent. Kemudian, kita membagi data menjadi data training (learning) dan data testing. Kali ini kita memilih cara pembagian data yang sederhana yaitu data testing adalah 30% dari total dataset.

Selanjutnya, untuk pemodelan dengan algoritma Decision Tree, jenis data harus numerik. Sedangkan kondisi dataset sekarang masih ada beberapa yang non-numerik,  yaitu Sex:male,female. Oleh karena itu atribute Sex harus ditransformasi menjadi nilai numerik. Dengan menggunakan tool **LabelEncoder** dari sklearn.preprocessing, kita dapat merubah secara otomatis atribute Sex menjadi male = 1, dan female = 0.

Selain itu variabel Age, Cabin dan Embarked juga non numerik, karena mengandung nilai NAN (not a number), sebagai ganti _missing value_. Kita perlu dapat juga menggunakan tool **LabelEncoder** seperti halnya variable Sex di atas. 


In [11]:
le = LabelEncoder()
dft['Sex'] =  le.fit_transform(dft['Sex'])
dft['Age'] = le.fit_transform(dft['Age'])
dft['Cabin'] =  le.fit_transform(dft['Cabin'])
dft['Embarked'] = le.fit_transform(dft['Embarked'])

In [12]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    int64  
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     891 non-null    int64  
 8   Embarked  891 non-null    int64  
dtypes: float64(1), int64(8)
memory usage: 62.8 KB


Sekarang kita melihat bahwa data sudah dalam tipe numerik.

In [13]:
#split dataset in features and target variable
feature_cols = ['Pclass','Sex','SibSp','Parch','Fare','Age','Cabin','Embarked']
X = dft[feature_cols] # Features
y = dft.Survived # Target variable

In [14]:
# Split dataset into training set and test set
# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 

#### Training: membangun model prediksi dengan Decision Tree

In [15]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

#### Evaluasi akurasi dari model yang dihasilkan: 


In [16]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7649253731343284
